In [ ]:
import copy
import os

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import trapezoid
from scipy.interpolate import RectBivariateSpline
from scipy.special import legendre

import SSLimPy.interface.sslimpy as sslimpy
from SSLimPy.LIMsurvey import covariance, power_spectrum
from scipy.signal import savgol_filter
from SSLimPy.utils.utils import *

In [ ]:
envkey = 'OMP_NUM_THREADS'
# Set this environment variable to the number of available cores in your machine,
# to get a fast execution of the Einstein Boltzmann Solver
os.environ[envkey] = str(7)

In [ ]:
cosmodict={"h":0.67,"Omegam":0.32,"Omegab":0.04,"As":2.1e-9, "mnu":0.06}
settings = {"code":"class",
            "do_RSD" : True,
            "nonlinearRSD" : True,
            "FoG_damp" : "ISTF_like",
            "halo_model_PS" : True,
            "Smooth_window" : False,
            "nk":100,
            "kmax": 50*u.Mpc**-1,
            "kmin": 1e-3*u.Mpc**-1,
            }
halopars = {
    "hmf_model":"sc_hmf",
    "bias_model": "b1",
}
Asslimpy = sslimpy.sslimpy(settings_dict=settings,
                           cosmopars=cosmodict)

In [ ]:
astro = Asslimpy.curent_astro
halo = astro.halomodel
cosmo = astro.cosmology
specs= astro.survey_specs

k = halo.k
M = halo.M

In [ ]:
Pcl = np.squeeze(astro.bhalo(k, 0.1))**2 * cosmo.matpow(k, 0.1)
Ps = astro.T_one_halo(k, 0.1)
Pobs = Pcl + Ps
plt.loglog(k, Pcl)
plt.loglog(k, Ps)
plt.loglog(k, Pobs)

In [ ]:
pobs = power_spectrum.PowerSpectra(astro)
kma = k<1*u.Mpc**-1
k = pobs.k
plt.loglog(k[kma], pobs.Pk_0bs[kma])
plt.loglog(k[kma], pobs.Pk_2bs[kma])
plt.loglog(k[kma], pobs.Pk_4bs[kma],c="green")
plt.loglog(k[kma], -pobs.Pk_4bs[kma],c="green", ls="--")

In [ ]:
k = pobs.k
z = pobs.z

pnl_ebs = cosmo.matpow(k, z, nonlinear=True, tracer="clustering")
pnl_hm = halo.P_halo(k, z)
pnl_hmb = halo.P_halo(k, z, bloating=True)
pnl_hmbd = halo.P_halo(k, z, bloating=True, P2h_sup=True)
pnl_hmbcd = halo.P_halo(k, z, bloating=True, P2h_sup=True, concB=True)


plt.loglog(k, cosmo.matpow(k,z, nonlinear=False), "k--")
plt.loglog(k, pnl_ebs, label="EBS")
plt.loglog(k, pnl_hm, label="HaloModel")
plt.loglog(k, pnl_hmb, label="HaloModel w/ bloating")
plt.loglog(k, pnl_hmbd, label="HaloModel w/ bloating \n + perturbative damping")
plt.loglog(k, pnl_hmbcd, label="HaloModel w/ bloating \n + perturbative damping \n + Bullock concentration")


plt.legend()
plt.xlabel(r"$k [\mathrm{Mpc}^{-1}]$")
plt.ylabel(r"$P [\mathrm{Mpc}^{3}]$")
